############################################ MODEL RUN TO CALCULATE PHES SIZE ################################################

The code calculates the minimum PHES size required to achieve the same energy generation results as the initial model run, where the introduced PHES capacity corresponds to the maximum possible   capacity installable in the region. Therefore, it is necessary to first run the model with maximum capacity availability.

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed Feb  7 16:38:11 2024

@author: baioc
"""
import calliope
import pandas as pd
import yaml
import os
import numpy as np
import time


try:
    calliope.set_log_level('INFO')
except:
    calliope.set_log_verbosity('Error')
    

DATA INPUT and DEFINE CHARACTHERISTICS OF MODEL

In [ ]:
#technologies present model
gen_techs = ['Hydro_Large', 'Hydro_Small', 'PV', 'PV_added', 'OCGT', 'CCGT', 'GST', 'HFO', 'Wind', 'Wind_added']  

#technologies added 
added_techs = ['PV_added', 'Wind_added']

#location of added technolgies
PV_added_loc = 'NGA_E'
Wind_added_loc = 'NGA_E'

#select deseried capacity values and shares for which the minimum PHES size needs to be calculated
shares = [0, 0.1, 0.2, 0.3, 0.4,0.5, 0.6, 0.7, 0.8, 0.9, 1]
capacity_values = list(range(0, 25001, 2500))

integration_strategies = ['PHES', 'PHES_TE' ]                                                            

# definition of integration strategies 
strategy_model = {
    'PHES': {'model_file': 'model_min_PH.yaml'},       
    'PHES_TE': {'model_file': 'model_min_PH_TE.yaml'} 
}

PHES CAPACITY MINIMIZATION MODEL RUN

In [ ]:
start_time = time.time()

for strategy in integration_strategies:
    
    for sh in shares:
        
        for cap in capacity_values:
            
            sh_name = int(sh*100)    
                        
            filepath =  f'Results/{strategy}/{sh_name}%Solar_{100-sh_name}%Wind/Added_renewables-{cap}MW'   
 
            energy_path = os.path.join(filepath,'results_carrier_prod.csv')
            capacity_path = os.path.join(filepath,'results_energy_cap.csv')     
            
            try:
                #take annual production of generation technologies
                energy=pd.read_csv(energy_path)
                energy=energy.loc[energy['techs'].isin(gen_techs)].groupby(['techs']).sum()
                            
            except FileNotFoundError:
                
                print(f"Missing maxium PHES capacity model run for:{filepath}")
                
                break
            
            #take capacity of unlimited model

            capacity=pd.read_csv(capacity_path)
            capacity=capacity.loc[capacity['techs'].isin(added_techs)]

            energy.reset_index(inplace=True)
        
        

            # ADDED RENEWAVLE CAPACITY
            
            with open('Model_config/Location_Constraints_PH_min.yaml', 'r') as file:
                Location_Constraints_PH_min = yaml.load(file, Loader=yaml.FullLoader)
            
            Location_Constraints_PH_min['locations'][Wind_added_loc]['techs']['Wind_added']['constraints']['energy_cap_equals']=capacity.loc[capacity['techs']=='Wind_added'].loc[:,'energy_cap'].sum().item()
            Location_Constraints_PH_min['locations'][PV_added_loc]['techs']['PV_added']['constraints']['energy_cap_equals']=capacity.loc[capacity['techs']=='PV_added'].loc[:,'energy_cap'].sum().item()
            with open('Model_config/Location_Constraints_PH_min_new.yaml', 'w') as outfile:
                yaml.dump(Location_Constraints_PH_min, outfile, default_flow_style=False)
                     
            
            # DEFINE PRODUCTION LIMITS AND DEFINE CONSTRAINTS
            
            #define production limitis            
            energy['carrier_prod_max']=energy['carrier_prod']*(1+0.025)
            energy['carrier_prod_min']=energy['carrier_prod']*(1-0.025)
                                                               
            #update overrides file                                                                  
            overrides_file_path = 'Model_config/Overrides_empty.yaml'

            with open(overrides_file_path, 'r') as file:
                overrides_new = yaml.load(file, Loader=yaml.FullLoader)
                                 
            for i in range(len(energy)):
            
                #minumum production constraint
                overrides_new['overrides']['Energy_constraint']['group_constraints'][f"{energy.iloc[i]['techs']}_Energy_min"]['carrier_prod_min']['power'] = energy.iloc[i]['carrier_prod_min'].item()
                #maximum production contraint
                overrides_new['overrides']['Energy_constraint']['group_constraints'][f"{energy.iloc[i]['techs']}_Energy_max"]['carrier_prod_max']['power'] = energy.iloc[i]['carrier_prod_max'].item()
                                
            with open('Model_config/Overrides_updated.yaml', 'w') as outfile:
                yaml.dump(overrides_new, outfile, default_flow_style=False)
                
            # MODEL RUN    



            model=calliope.Model(strategy_model[strategy]['model_file'], scenario='Energy_constraint')
            
            model.run()
            
            model.to_csv(f'Results/Minimized_PHES/{strategy}/{sh_name}%Solar_{100-sh_name}%Wind_PHES_min/Added_renewables-{cap}MW')


end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time:", elapsed_time, "seconds")
